# --- Day 14: Parabolic Reflector Dish ---
You reach the place where all of the mirrors were pointing: a massive parabolic reflector dish attached to the side of another large mountain.

The dish is made up of many small mirrors, but while the mirrors themselves are roughly in the shape of a parabolic reflector dish, each individual mirror seems to be pointing in slightly the wrong direction. If the dish is meant to focus light, all it's doing right now is sending it in a vague direction.

This system must be what provides the energy for the lava! If you focus the reflector dish, maybe you can go where it's pointing and use the light to fix the lava production.

Upon closer inspection, the individual mirrors each appear to be connected via an elaborate system of ropes and pulleys to a large metal platform below the dish. The platform is covered in large rocks of various shapes. Depending on their position, the weight of the rocks deforms the platform, and the shape of the platform controls which ropes move and ultimately the focus of the dish.

In short: if you move the rocks, you can focus the dish. The platform even has a control panel on the side that lets you tilt it in one of four directions! The rounded rocks (O) will roll when the platform is tilted, while the cube-shaped rocks (#) will stay in place. You note the positions of all of the empty spaces (.) and rocks (your puzzle input). For example:
```
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
```
Start by tilting the lever so all of the rocks will slide north as far as they will go:
```
OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....
```
You notice that the support beams along the north side of the platform are damaged; to ensure the platform doesn't collapse, you should calculate the total load on the north support beams.

The amount of load caused by a single rounded rock (O) is equal to the number of rows from the rock to the south edge of the platform, including the row the rock is on. (Cube-shaped rocks (#) don't contribute to load.) So, the amount of load caused by each rock in each row is as follows:
```
OOOO.#.O.. 10
OO..#....#  9
OO..O##..O  8
O..#.OO...  7
........#.  6
..#....#.#  5
..O..#.O.O  4
..O.......  3
#....###..  2
#....#....  1
```
The total load is the sum of the load caused by all of the rounded rocks. In this example, the total load is 136.

Tilt the platform so that the rounded rocks all roll north. Afterward, what is the total load on the north support beams?


## --- Part Two ---
The parabolic reflector dish deforms, but not in a way that focuses the beam. To do that, you'll need to move the rocks to the edges of the platform. Fortunately, a button on the side of the control panel labeled "spin cycle" attempts to do just that!

Each cycle tilts the platform four times so that the rounded rocks roll north, then west, then south, then east. After each tilt, the rounded rocks roll as far as they can before the platform tilts in the next direction. After one cycle, the platform will have finished rolling the rounded rocks in those four directions in that order.

Here's what happens in the example above after each of the first few cycles:

After 1 cycle:
```
.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....
```
After 2 cycles:
```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O
```
After 3 cycles:
```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O
```
This process should work if you leave it running long enough, but you're still worried about the north support beams. To make sure they'll survive for a while, you need to calculate the total load on the north support beams after 1000000000 cycles.

In the above example, after 1000000000 cycles, the total load on the north support beams is 64.

Run the spin cycle for 1000000000 cycles. Afterward, what is the total load on the north support beams?

In [1]:
from pathlib import Path
import os

yr = 2023
d = 14

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import numpy as np
def format_input(inp):
  formatted_input = []
  for l in inp.split('\n'):
    formatted_input.append(list(l))
  return np.array(formatted_input, dtype=str)

In [3]:
def roll_single(arr, rev=False):
  '''
  Roll the rocks in a single row/column
  '''
  roll_to_loc = 0

  if rev:
    arr = list(reversed(arr))
  for i in range(len(arr)):
    if arr[i]=='#':
      roll_to_loc=i+1
    elif arr[i]=='O':
      arr[i] = '.'
      arr[roll_to_loc] = 'O'
      roll_to_loc += 1
  if rev:
    return list(reversed(arr))
  else:
    return arr

In [4]:
def roll(arr, dir='up', cp=True):
  '''
  Roll all the rocks
  '''
  if cp:
    import copy
    arr = copy.deepcopy(arr)
  if dir == 'up':
    for i in range(arr.shape[1]):
      arr[:,i] = roll_single(arr[:,i])
  elif dir == 'down':
    for i in range(arr.shape[1]):
      arr[:,i] = roll_single(arr[:,i], rev=True)
  elif dir == 'left':
    for i in range(arr.T.shape[0]):
      arr.T[:,i] = roll_single(arr.T[:,i])
  elif dir == 'right':
    for i in range(arr.T.shape[0]):
      arr.T[:,i] = roll_single(arr.T[:,i], rev=True)
  return arr

In [5]:
def calculate_load(arr):
  '''
  Calculate the load of the current rock configuration
  '''
  s = 0
  for i in range(arr.shape[0]):
    s += (sum([x=='O' for x in arr[arr.shape[0]-1-i]])) * (i+1)
  return s

In [6]:
def roll_north_and_calculate_load(arr):
  return calculate_load(roll(arr))

In [7]:
def spin_cycle(arr, cp=True):
  '''
  Do a full spin cycle
  '''
  return roll(roll(roll(roll(arr, dir='up', cp=cp), dir='left', cp=False), dir='down', cp=False), dir='right', cp=False)

In [8]:
def arr_to_str(arr):
  '''
  Convert the array that represents the platform to a string
  '''
  return ''.join([''.join(x) for x in arr])

def arr_to_hash(arr):
  '''
  Hash the platform array with arr_to_str() as an intermediate step
  '''
  return hash(arr_to_str(arr))

In [9]:
def find_spin_cycle_loop(arr, cp=True):
  '''
  Run the spin cycle until you get back
  to a configuration you've seen before

  return the hashes of the array after each cycle,
  as well as the arrays themselves
  '''
  if cp:
    import copy
    arr = copy.deepcopy(arr)
  hashes = [arr_to_hash(arr)]
  hash_2_arr = {arr_to_hash(arr): arr}
  while(len(set(hashes))==len(hashes)):
    arr = spin_cycle(arr, cp=True)
    hashes.append(arr_to_hash(arr))
    hash_2_arr[arr_to_hash(arr)] = arr
  return hashes, hash_2_arr


In [10]:
def spin_cycle_and_calculate_load(arr, n, cp=True):
  '''
  Figure out how long it takes to get into a cycle
  and use some modular arithmetic to determine what the
  configuration will be like after n spin cycles

  return the load of that configuration
  '''
  if cp:
    import copy
    arr = copy.deepcopy(arr)
  hashes, hash_2_arr = find_spin_cycle_loop(arr, cp=False)
  n_before_loop = hashes.index(hashes[-1]) - 1
  loop_size = len(hashes) - n_before_loop - 2
  n_hash = hashes[(n_before_loop + (n-n_before_loop)%loop_size)]
  n_arr = hash_2_arr[n_hash]
  return calculate_load(hash_2_arr[n_hash])

In [11]:
import time

t = time.time()

formatted_input = format_input(inp)

print(roll_north_and_calculate_load(formatted_input))
print(spin_cycle_and_calculate_load(formatted_input, 1000000000))

print('\nRUNTIME: ', time.time()-t)

112048
105606

RUNTIME:  3.393922805786133
